In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[3],3a
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}","{'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}"
nv,-,1000
tsfc,257,257


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.200939                    1  0.126876
0.000750        2  0.228723                    2  0.152994
0.001052        3  0.309656                    3  0.213983
0.001476        4  0.398929                    4  0.282248
0.002070        5  0.494751                    5  0.357172
0.002904        6  0.595432                    6  0.438305
0.004074        7  0.698397                    7  0.527983
0.005714        8  0.800126                    8  0.624955
0.008015        9  0.896308                    9  0.725171
0.011243       10  0.981622                   10  0.825118
0.015771       11  1.050275                   11  0.920087
0.022122       12  1.097021                   12  1.003604
0.031031       13  1.118469                   13  1.066317
0.043528       14  1.090461                   14  1.078826
0.061057       15  0.998875                   15  1.017097
0.085645       16  0.882264                   16  0.901697
0.120136       17  0.774700                   17  0.763393
0.168516       18  0.686618                   18  0.629737
0.236378       19  0.625128                   19  0.532887
0.331549       20  0.596931                   20  0.496463
0.465100       21  0.615525                   21  0.530311
0.652400       22  0.559206                   22  0.512727
0.915100       23  0.417947                   23  0.411045
1.283650       24  0.312792                   24  0.322175
1.800600       25  0.245709                   25  0.256372
2.525700       26  0.194792                   26  0.203892
3.542800       27  0.151088                   27  0.163437
4.969550       28  0.120195                   28  0.137863
6.970850       29  0.104585                   29  0.127358
9.778100       30  0.092568                   30  0.115741
13.715850      31  0.078717                   31  0.096570
19.239350      32  0.063914                   32  0.072772
26.987250      33  0.057151                   33  0.057862
37.855300      34  0.058665                   34  0.054560
53.100050      35  0.059094                   35  0.056814
73.887500      36  0.058473                   36  0.062694
97.662500      37  0.057146                   37  0.065291
121.437500     38  0.052769                   38  0.061138
145.212500     39  0.047673                   39  0.054251
168.987500     40  0.043315                   40  0.047832
192.762500     41  0.039009                   41  0.041736
216.537500     42  0.034126                   42  0.035582
240.312500     43  0.028561                   43  0.028874
264.087500     44  0.021656                   44  0.020860
287.862500     45  0.012629                   45  0.010930
311.637500     46  0.010395                   46  0.008184
335.412500     47  0.020523                   47  0.018128
359.187500     48  0.034017                   48  0.031610
382.962500     49  0.049134                   49  0.046052
406.737500     50  0.066663                   50  0.062232
430.512500     51  0.084602                   51  0.078758
454.287500     52  0.102164                   52  0.095321
478.062500     53  0.118805                   53  0.111402
501.837500     54  0.134313                   54  0.125859
525.612500     55  0.148607                   55  0.138267
549.387500     56  0.161567                   56  0.149525
573.162500     57  0.173121                   57  0.160425
596.937500     58  0.183029                   58  0.170814
620.712500     59  0.191065                   59  0.179671
644.487500     60  0.197034                   60  0.186479
668.262500     61  0.200870                   61  0.190716
692.037500     62  0.202717                   62  0.192691
715.812500     63  0.202846                   63  0.192981
739.587500     64  0.201727                   64  0.192242
763.362500     65  0.199966                   65  0.191087
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -19.704359   0.000000 -19.704359 -19.778943  4.523207e-08   
0.000624    2     -19.704394   0.000050 -19.704344 -19.778971  3.762316e-05   
0.000876    3     -19.704408   0.000071 -19.704337 -19.778982  5.347297e-05   
0.001229    4     -19.704428   0.000103 -19.704325 -19.778998  7.790425e-05   
0.001723    5     -19.704454   0.000153 -19.704301 -19.779019  1.155055e-04   
0.002417    6     -19.704489   0.000229 -19.704260 -19.779047  1.731826e-04   
0.003391    7     -19.704536   0.000345 -19.704192 -19.779085  2.613448e-04   
0.004757    8     -19.704598   0.000519 -19.704079 -19.779134  3.962893e-04   
0.006672    9     -19.704678   0.000781 -19.703897 -19.779199  6.023637e-04   
0.009359    10    -19.704780   0.001168 -19.703612 -19.779280  9.147715e-04   
0.013128    11    -19.704905   0.001731 -19.703174 -19.779381  1.383884e-03   
0.018415    12    -19.705055   0.002539 -19.702516 -19.779501  2.079776e-03   
0.025830    13    -19.705226   0.003674 -19.701552 -19.779635  3.095640e-03   
0.036232    14    -19.705404   0.005230 -19.700174 -19.779771  4.545815e-03   
0.050823    15    -19.705570   0.007281 -19.698289 -19.779893  6.533195e-03   
0.071291    16    -19.705720   0.009853 -19.695868 -19.779995  9.100957e-03   
0.100000    17    -19.705842   0.012975 -19.692867 -19.780059  1.223217e-02   
0.140271    18    -19.705889   0.016718 -19.689172 -19.780042  1.585711e-02   
0.196760    19    -19.705778   0.021201 -19.684577 -19.779866  1.989518e-02   
0.275997    20    -19.705350   0.026640 -19.678709 -19.779391  2.442219e-02   
0.387100    21    -19.704261   0.033408 -19.670853 -19.778317  2.988259e-02   
0.543100    22    -19.701519   0.042040 -19.659479 -19.775760  3.712660e-02   
0.761700    23    -19.696762   0.051764 -19.644998 -19.771336  4.598106e-02   
1.068500    24    -19.691158   0.061349 -19.629809 -19.766023  5.560804e-02   
1.498800    25    -19.685283   0.071418 -19.613865 -19.760329  6.633782e-02   
2.102400    26    -19.679196   0.082899 -19.596297 -19.754303  7.864446e-02   
2.949000    27    -19.673094   0.096332 -19.576762 -19.748141  9.293164e-02   
4.136600    28    -19.667661   0.112154 -19.555507 -19.742425  1.102112e-01   
5.802500    29    -19.663487   0.131699 -19.531788 -19.737653  1.326472e-01   
8.139200    30    -19.660313   0.157473 -19.502839 -19.733538  1.637877e-01   
11.417000   31    -19.658307   0.191410 -19.466897 -19.730282  2.054769e-01   
16.014700   32    -19.659022   0.234997 -19.424025 -19.729585  2.573796e-01   
22.464000   33    -19.666156   0.290959 -19.375197 -19.735470  3.188662e-01   
31.510500   34    -19.682185   0.368233 -19.313952 -19.750645  3.960540e-01   
44.200100   35    -19.706317   0.480548 -19.225769 -19.773935  5.013653e-01   
62.000000   36    -19.740216   0.639049 -19.101167 -19.807247  6.544831e-01   
85.775000   37    -19.784560   0.848073 -18.936488 -19.851661  8.754811e-01   
109.550000  38    -19.827966   1.052419 -18.775547 -19.895703  1.103422e+00   
133.325000  39    -19.873644   1.246711 -18.626933 -19.942353  1.322272e+00   
157.100000  40    -19.925574   1.432904 -18.492670 -19.995407  1.528131e+00   
180.875000  41    -19.984614   1.613931 -18.370683 -20.055656  1.723103e+00   
204.650000  42    -20.051830   1.791008 -18.260822 -20.124261  1.909262e+00   
228.425000  43    -20.128452   1.963739 -18.164713 -20.203070  2.088292e+00   
252.200000  44    -20.216624   2.132348 -18.084276 -20.294592  2.261141e+00   
275.975000  45    -20.320791   2.297506 -18.023286 -20.403511  2.428815e+00   
299.750000  46    -20.450006   2.462286 -17.987719 -20.538571  2.594659e+00   
323.525000  47    -20.603526   2.645082 -17.958444 -20.697899  2.777038e+00   
347.300000  48    -20.767431   2.866786 -17.900645 -20.866477  2.996675e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')